In [33]:
import pandas as pd
import numpy as np
import json

In [34]:
with open("ccpp_parsed.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [35]:
with open("ccpp_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [36]:
dfs_ebs["connections"][["e_T", "e_M"]] = dfs_ebs["connections"][["e_T", "e_M"]].round(6)
dfs_tespy["connections"][["e_T", "e_M"]] = dfs_tespy["connections"][["e_T", "e_M"]].round(6)

In [37]:
columns = ["m", "p", "T", "e_T", "e_M"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ) / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)
diff_to_ebs = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ).abs() / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)

In [38]:
diff_to_ebs

,m,p,T,e_T,e_M
20,0.000585,0.000000e+00,1.608973e-09,1.310402e-07,0.000021
4,0.000430,0.000000e+00,1.789509e-07,1.259134e-03,0.001017
10,0.001123,0.000000e+00,9.270300e-05,5.949741e-05,0.000014
13,0.001171,0.000000e+00,7.429923e-16,3.210844e-05,0.000182
8,0.000430,0.000000e+00,2.937235e-05,6.349441e-04,NaN
7,0.000430,4.542686e-10,3.798476e-08,5.223480e-04,0.000148
1,0.000427,0.000000e+00,0.000000e+00,NaN,NaN
2,0.000427,0.000000e+00,1.073469e-03,7.146881e-03,0.001048
9,0.000585,0.000000e+00,8.690575e-05,6.954548e-05,0.000006
12,0.001171,0.000000e+00,9.270300e-05,5.949741e-05,0.000014


In [39]:
diff_to_ebs[diff_to_ebs > 1e-3].dropna(how="all")

,m,p,T,e_T,e_M
4,NaN,NaN,NaN,0.001259,0.001017
10,0.001123,NaN,NaN,NaN,NaN
13,0.001171,NaN,NaN,NaN,NaN
2,NaN,NaN,0.001073,0.007147,0.001048
12,0.001171,NaN,NaN,NaN,NaN
17,0.001171,NaN,NaN,NaN,NaN
16,0.001171,NaN,NaN,NaN,NaN
15,0.001235,NaN,NaN,NaN,NaN
14,0.001235,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.010478


In [45]:
# Load the CSV files
tespy_results = pd.read_csv("ccpp_components_tespy.csv", index_col=0)
ebsilon_results = pd.read_csv("ccpp_components_ebsilon.csv", index_col=0)

# Step 1: Normalize the component names in Tespy
# Group "drum" and "drum pump" into "EVA"
tespy_results.loc[tespy_results["Component"].isin(["drum", "drum pump"]), "Component"] = "EVA"

# Group "dea steam inlet valve" into "DEA"
tespy_results.loc[tespy_results["Component"] == "dea steam inlet valve", "Component"] = "DEA"

# Group "MIX" into "DEA"
ebsilon_results.loc[ebsilon_results["Component"] == "MIX", "Component"] = "DEA"

# Aggregate the values by summing them up
tespy_results = tespy_results.groupby("Component", as_index=False).sum(numeric_only=True)
ebsilon_results = ebsilon_results.groupby("Component", as_index=False).sum(numeric_only=True)

# Step 2: Merge DataFrames on Component name
merged_df = pd.merge(ebsilon_results, tespy_results, on="Component", suffixes=("_Ebsilon", "_Tespy"))

# Step 3: Compute Differences
columns_to_compare = ["E_F [kW]", "E_P [kW]", "E_D [kW]", "E_L [kW]", "ε [%]", "y [%]", "y* [%]"]
for col in columns_to_compare:
    merged_df[f"Diff_{col}"] = merged_df[f"{col}_Tespy"] - merged_df[f"{col}_Ebsilon"]

# Step 4: Select only the relevant columns
diff_columns = [col for col in merged_df.columns if col.startswith("Diff_E_D") or col.endswith("E_D [kW]_Ebsilon") or col.endswith("E_D [kW]_Tespy")]
diff_df = merged_df[["Component"] + diff_columns]
diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100

# Display the filtered DataFrame
diff_df

C:\Users\sergiotomasinelli\AppData\Local\Temp\ipykernel_1296\3097275549.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100


,Component,E_D [kW]_Ebsilon,E_D [kW]_Tespy,Diff_E_D [kW],Relative error [%]
0,CC,195766.602870,195482.213037,-284.389833,-0.145481
1,COMP,10945.496471,10950.717811,5.221340,0.047680
2,COND,3717.826764,3713.600960,-4.225804,-0.113793
3,DEA,3184.745810,3181.242206,-3.503604,-0.110133
4,ECO,1475.374949,1474.425924,-0.949026,-0.064366
5,EVA,11648.296333,11647.452712,-0.843622,-0.007243
6,GT,15969.551601,15980.073451,10.521850,0.065844
7,HC,20.937228,20.936112,-0.001117,-0.005333
8,PUMP1,7.087864,7.079578,-0.008286,-0.117042
9,PUMP2,57.850131,57.816224,-0.033907,-0.058645
